In [44]:
import pandas as pd

Get Seasons 


In [45]:
def get_seasons():
    start_year = 2000
    end_year = 2022
    start_season = 1
    end_season = 2

    year_season_list = []

    for year in range(start_year, end_year + 1):
        # Calculate the season for the current year
        season = start_season + (year - start_year)
        # Format the year and season as strings and append to the list
        year_season_list.append(f"{year}_{season:02d}")
    return year_season_list

season_list = get_seasons()
season_list

['2000_01',
 '2001_02',
 '2002_03',
 '2003_04',
 '2004_05',
 '2005_06',
 '2006_07',
 '2007_08',
 '2008_09',
 '2009_10',
 '2010_11',
 '2011_12',
 '2012_13',
 '2013_14',
 '2014_15',
 '2015_16',
 '2016_17',
 '2017_18',
 '2018_19',
 '2019_20',
 '2020_21',
 '2021_22',
 '2022_23']

Merge Data Across Seasons


In [46]:
# Group all the datagrams from our data
def mergeData (data_folder, file_prefix):
#Group advanced stats
    combined_data = pd.DataFrame()

    for s in season_list:
        loc = f'data/{data_folder}/{s}_{file_prefix}.csv'
        data = pd.read_csv(loc)
        data['season'] = s
        combined_data = pd.concat([combined_data, data], ignore_index=True)
    return combined_data

advanced_stats_df = mergeData("advanced_stats", "playerstats_advanced")
player_stats_df = mergeData("player_stats", "playerstats")
team_standings_df = mergeData("team_standings", "teamstandings")


Drop any columns that overlap, as data is quite similar across the various types of advanced stats, player stats and team standings. However, keep the columns that will allow the merging later

In [47]:
columns_advanced_stats_df = set(advanced_stats_df.columns)
columns_player_stats_df = set(player_stats_df.columns)
columns_team_standings_df = set(team_standings_df.columns)

common_columns_advanced_player = columns_advanced_stats_df.intersection(columns_player_stats_df)
common_columns_player_team = columns_player_stats_df.intersection(columns_team_standings_df)
common_columns_advanced_team = columns_advanced_stats_df.intersection(columns_team_standings_df)

if common_columns_advanced_player: #common between advanced and player
    print(common_columns_advanced_player)
    advanced_stats_df.drop(columns= ['Age', 'Pos', 'G', 'MP', 'Rk'], axis = 1, inplace = True) #Keep player. TM (team) and season to enable the merging of data

if common_columns_player_team and common_columns_player_team and len(common_columns_player_team) > 1: #player and team standings
    print(common_columns_player_team)

if common_columns_advanced_team and common_columns_advanced_team and len(common_columns_advanced_team) > 1: #player and advanced team
    print(common_columns_advanced_team)
print(advanced_stats_df.columns)

{'Age', 'Pos', 'Rk', 'Tm', 'Player', 'MP', 'Player-additional', 'G', 'season'}
Index(['Player', 'Tm', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS',
       'WS', 'WS/48', 'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Player-additional', 'season'],
      dtype='object')


Remove any columns that may not be useful. 

In [50]:
#Remove Null's for advanced_stats
null_adv = advanced_stats_df.isnull().sum()
print(null_adv) 
#Key null columns are Unamed:19 and Unamed:24 
advanced_stats_df.drop(columns= ['Unnamed: 19', 'Unnamed: 24'], axis = 1, inplace = True) #Keep player. TM (team) and season to enable the merging of data


Player                   0
Tm                       0
PER                      5
TS%                     79
3PAr                    83
FTr                     83
ORB%                     5
DRB%                     5
TRB%                     5
AST%                     5
STL%                     5
BLK%                     5
TOV%                    66
USG%                     5
Unnamed: 19          13927
OWS                      0
DWS                      0
WS                       0
WS/48                    5
Unnamed: 24          13927
OBPM                     0
DBPM                     0
BPM                      0
VORP                     0
Player-additional        0
season                   0
dtype: int64


In [52]:
#Remove Null's for player_stats
null_player_stats = player_stats_df.isnull().sum()
print(null_player_stats) #No signficiant columns with null values or high count of nulls

Rk                      0
Player                  0
Pos                     0
Age                     0
Tm                      0
G                       0
GS                      0
MP                      0
FG                      0
FGA                     0
FG%                    83
3P                      0
3PA                     0
3P%                  1930
2P                      0
2PA                     0
2P%                   152
eFG%                   83
FT                      0
FTA                     0
FT%                   716
ORB                     0
DRB                     0
TRB                     0
AST                     0
STL                     0
BLK                     0
TOV                     0
PF                      0
PTS                     0
Player-additional       0
season                  0
dtype: int64


In [55]:
#Remove Null's for team standings
null_team_standings = team_standings_df.isnull().sum()
print(null_team_standings) #No signficiant columns with null values or high count of nulls

Unnamed: 0        0
Unnamed: 1        0
Unnamed: 2       30
Place             0
Place.1           0
Conference        0
Conference.1      0
Division          0
Division.1        0
Division.2        0
Division.3        0
All-Star          0
All-Star.1        0
Margin            0
Margin.1          0
Month            47
Month.1           0
Month.2           0
Month.3           0
Month.4           0
Month.5          31
Month.6         138
season            0
Division.4      120
Division.5      120
Month.7         686
dtype: int64


Remove any unecessary columns

In [58]:
# Based on an anlyasis of the data, the rk column can be removed as its a numercial value that provided no value and could confuse the model
# Remove rank for player_stats as well, team rank does matter as we can guage who the best team is
player_stats_df.drop(columns= ['Rk'], axis = 1, inplace = True) 



KeyError: "['Rk'] not found in axis"

Merge Data into one large dataframe across both advanced and player stats

In [60]:
#Left join, must keep player_stats and only merge data if it only corresponds to one row
total_data_df = player_stats_df.merge(advanced_stats_df, on=['Player', 'season', 'Tm'], how ='left', validate='1:1')
total_data_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Player-additional_y
0,Mahmoud Abdul-Rauf,PG,31,VAN,41,0,11.9,2.9,6.0,0.488,...,26.8,0.7,0.0,0.7,0.070,1.1,-2.1,-1.0,0.1,abdulma02
1,Tariq Abdul-Wahad,SG,26,DEN,29,12,14.5,1.5,3.8,0.387,...,17.1,-0.7,0.3,-0.4,-0.049,-5.6,-0.3,-5.8,-0.4,abdulta01
2,Shareef Abdur-Rahim,SF,24,VAN,81,81,40.0,7.5,15.8,0.472,...,24.6,5.6,2.9,8.5,0.126,1.7,0.1,1.8,3.1,abdursh01
3,Cory Alexander,PG,27,ORL,26,0,8.7,0.7,2.2,0.321,...,17.3,-0.5,0.3,-0.1,-0.031,-7.2,1.2,-6.0,-0.2,alexaco01
4,Courtney Alexander,PG,23,TOT,65,24,21.3,3.7,8.8,0.417,...,23.2,0.3,0.5,0.8,0.027,-2.4,-1.9,-4.3,-0.8,alexaco02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13922,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,...,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,youngth01
13923,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,...,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,youngtr01
13924,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,...,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,yurtsom01
13925,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,...,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,zelleco01
